<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-Sample-data:" data-toc-modified-id="Import-Sample-data:-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import Sample data:</a></span></li><li><span><a href="#Setup-multivariate-linear-models" data-toc-modified-id="Setup-multivariate-linear-models-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Setup multivariate linear models</a></span></li><li><span><a href="#Confirm-identical-coef/params-for-statsmodels-vs-sklearn:" data-toc-modified-id="Confirm-identical-coef/params-for-statsmodels-vs-sklearn:-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Confirm identical coef/params for statsmodels vs sklearn:</a></span><ul class="toc-item"><li><span><a href="#Validate-calculation-of-squared-sum-of-residuals-(SSR):" data-toc-modified-id="Validate-calculation-of-squared-sum-of-residuals-(SSR):-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Validate calculation of squared sum of residuals (SSR):</a></span><ul class="toc-item"><li><span><a href="#SSR-(residual-sum-of-squares)-vs-MSE-(mean-squared-error):" data-toc-modified-id="SSR-(residual-sum-of-squares)-vs-MSE-(mean-squared-error):-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>SSR (residual sum of squares) vs MSE (mean squared error):</a></span></li></ul></li></ul></li><li><span><a href="#Confirm-class-attributes-are-accurate-for-multivariate-models-created-with-or-without-a-formula-or-constant:" data-toc-modified-id="Confirm-class-attributes-are-accurate-for-multivariate-models-created-with-or-without-a-formula-or-constant:-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Confirm class attributes are accurate for multivariate models created with or without a formula or constant:</a></span></li><li><span><a href="#Validate-statistical-calculations-with-univariate-models:" data-toc-modified-id="Validate-statistical-calculations-with-univariate-models:-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Validate statistical calculations with univariate models:</a></span><ul class="toc-item"><li><span><a href="#Validate-log-likelihood-calculation:" data-toc-modified-id="Validate-log-likelihood-calculation:-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Validate log likelihood calculation:</a></span></li><li><span><a href="#Validate-AIC-calculation" data-toc-modified-id="Validate-AIC-calculation-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Validate AIC calculation</a></span></li></ul></li><li><span><a href="#Validate-model-stats-against-R:" data-toc-modified-id="Validate-model-stats-against-R:-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Validate model stats against R:</a></span></li><li><span><a href="#Validate-implementation-of-F-test" data-toc-modified-id="Validate-implementation-of-F-test-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Validate implementation of F test</a></span><ul class="toc-item"><li><span><a href="#Calculate-the-F-statistic-using-residual-sum-of-squares:" data-toc-modified-id="Calculate-the-F-statistic-using-residual-sum-of-squares:-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Calculate the F statistic using residual sum of squares:</a></span></li><li><span><a href="#Compare-with-sm.stats.anova_lm()" data-toc-modified-id="Compare-with-sm.stats.anova_lm()-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Compare with sm.stats.anova_lm()</a></span></li><li><span><a href="#Validate-F-test-works-with-multivariate-models:" data-toc-modified-id="Validate-F-test-works-with-multivariate-models:-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>Validate F test works with multivariate models:</a></span></li><li><span><a href="#Compare-F-test-calculation-with-R:" data-toc-modified-id="Compare-F-test-calculation-with-R:-7.4"><span class="toc-item-num">7.4&nbsp;&nbsp;</span>Compare F test calculation with R:</a></span></li></ul></li><li><span><a href="#How-to-implement-MANOVA" data-toc-modified-id="How-to-implement-MANOVA-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>How to implement MANOVA</a></span><ul class="toc-item"><li><span><a href="#Compare-statsmodels-MANOVA-with-R:" data-toc-modified-id="Compare-statsmodels-MANOVA-with-R:-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>Compare statsmodels MANOVA with R:</a></span></li></ul></li><li><span><a href="#Forward-stepwise-model-selection" data-toc-modified-id="Forward-stepwise-model-selection-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Forward stepwise model selection</a></span></li></ul></div>

# Multivariate OLS
Author : Meaghan Flagg  
Purpose : implementing and testing summary statistics and F-test for multivariate OLS

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import sklearn.linear_model as lm
from sklearn.metrics import r2_score, mean_squared_error
import statsmodels.api as sm
import statsmodels.multivariate.manova as manova

In [2]:
import statsmodels.multivariate.multivariate_ols # autoreload doesn't work with import x as y

In [3]:
%load_ext autoreload
%autoreload 2

## Import Sample data:

In [4]:
cars = sm.datasets.get_rdataset("mtcars").data
cars.head()

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


## Setup multivariate linear models
Model 1/4 mile time and hp as a function of mpg, cylinders, and displacement:  
` qsec + hp ~ mpg + cyl + disp`

In [5]:
# using statsmodels multivariate OLS
MOD = statsmodels.multivariate.multivariate_ols._MultivariateOLS.from_formula(
    formula="qsec + hp ~ mpg + cyl + disp", data=cars)
mod = MOD.fit()

In [6]:
# Model constructed without formula:
X = cars[['mpg','cyl','disp']]
Y = cars[['qsec','hp']]
noformula_MOD = statsmodels.multivariate.multivariate_ols._MultivariateOLS(endog=Y, #exog=X)
                                                exog=sm.add_constant(X))
noformula_mod = noformula_MOD.fit()

In [7]:
# Model without constant:
X = cars[['mpg','cyl','disp']]
Y = cars[['qsec','hp']]
noConst_MOD = statsmodels.multivariate.multivariate_ols._MultivariateOLS(endog=Y, exog=X)

noConst_mod = noConst_MOD.fit()

## Confirm identical coef/params for statsmodels vs sklearn:
I'll use p1, p2, p3 to indicate parameters (X variables) 1, 2, 3

`_Multivariate_OLS.fit()` returns tuple of: `(params, df_resid, inv_cov, sscpr)`  
What are each of these?  
`params : calculated coefficient values for X vars (plus constant) for each Y var`  
`df_resid : residual degrees of freedom. n - p - 1, if a constant is present. n - p if a constant is not included.`  
`inv_cov : inverse covariance matrix`  
`sscpr : possibly sum of squares and cross-products decomposition for the regression`  
* the diagonals are sum of squares (for each Y var), corners are cross-products
* access ssr via: `np.diagonal(mod._fittedmod[3])`

In [8]:
# sklearn
X = cars[['mpg','cyl','disp']]
Y = cars[['qsec','hp']]

# fitting intercept: can either add constant to X array, or set fit_intercept=True in regression call.
reg = lm.LinearRegression(fit_intercept=True, normalize=True).fit(X,Y)

In [9]:
# statsmodels:
print(type(mod._fittedmod[0]))
print(mod._fittedmod[0])
# shape
#            Y1   Y2
# intercept   .    .
# p1          .    .
# p2          .    .
# p3          .    .

<class 'numpy.ndarray'>
[[ 2.45534817e+01  4.92951014e+01]
 [-5.30029521e-02 -2.35788925e+00]
 [-1.15824380e+00  2.07718985e+01]
 [ 6.61733652e-03  7.03780053e-02]]


In [10]:
# sklearn
print("coefs")
print(type(reg.coef_))
print(reg.coef_)
# shape:
#    p1   p2   p3
# Y1  .   .    .
# Y2  .   .    .

print("\nintercept")
print(type(reg.intercept_))
print(reg.intercept_)
# shape:
# Y1  Y2

coefs
<class 'numpy.ndarray'>
[[-5.30029521e-02 -1.15824380e+00  6.61733652e-03]
 [-2.35788925e+00  2.07718985e+01  7.03780053e-02]]

intercept
<class 'numpy.ndarray'>
[24.55348166 49.29510142]


1. coefficients are identical
1. Intercepts are identical  

Great.
### Validate calculation of squared sum of residuals (SSR):
access array of ssrs by `np.diagonal(mod._fittedmod[3])`

In [11]:
np.diagonal(mod._fittedmod[3])

array([   58.35231457, 41982.82756707])

Options for aggregating ssr for each Y var:  
1. uniform average
2. weighted average

I will follow the convention in `sklearn.metrics.mean_squared_error`:

In [12]:
print(mean_squared_error.__doc__)

Mean squared error regression loss

    Read more in the :ref:`User Guide <mean_squared_error>`.

    Parameters
    ----------
    y_true : array-like of shape (n_samples,) or (n_samples, n_outputs)
        Ground truth (correct) target values.

    y_pred : array-like of shape (n_samples,) or (n_samples, n_outputs)
        Estimated target values.

    sample_weight : array-like of shape (n_samples,), optional
        Sample weights.

    multioutput : string in ['raw_values', 'uniform_average']                 or array-like of shape (n_outputs)
        Defines aggregating of multiple output values.
        Array-like value defines weights used to average errors.

        'raw_values' :
            Returns a full set of errors in case of multioutput input.

        'uniform_average' :
            Errors of all outputs are averaged with uniform weight.

    squared : boolean value, optional (default = True)
        If True returns MSE value, if False returns RMSE value.

    Returns
 

#### SSR (residual sum of squares) vs MSE (mean squared error):
$ ssr = MSE \times nobs $  
where `nobs = number of observations`


In [13]:
np.mean(np.diagonal(mod._fittedmod[3]))

21020.589940823113

In [14]:
# test my implementation of ssr method in _MultivariateOLSResults class:
mod.ssr()

21020.589940823113

In [15]:
# confirm with sklearn mean_squared_error:
Yhat = reg.predict(X)
nobs=Y.shape[0]
RSS_all = mean_squared_error(Y, Yhat, multioutput='raw_values') * nobs
RSS_mean = mean_squared_error(Y, Yhat, multioutput='uniform_average') * nobs
print(RSS_all)
print(RSS_mean)

[   58.35231457 41982.82756708]
21020.589940825048


Excellent, everything lines up.

## Confirm class attributes are accurate for multivariate models created with or without a formula or constant:
**Note on how some attributes are calculated:**  
`_Multivariate_OLS.fit()` returns tuple of: `(params, df_resid, inv_cov, sscpr)`  
What are each of these?  
`params : calculated coefficient values for X vars (plus constant) for each Y var`  
`df_resid : residual degrees of freedom. n - p - 1, if a constant is present. n - p if a constant is not included.`  
`inv_cov : inverse covariance matrix`  
`sscpr : possibly sum of squares and cross-products decomposition for the regression`  
* the diagonals are sum of squares (for each Y var), corners are cross-products
* access ssr via: `np.diagonal(mod._fittedmod[3])`

In [16]:
print(type(mod))
print(type(noformula_mod))

<class 'statsmodels.multivariate.multivariate_ols._MultivariateOLSResults'>
<class 'statsmodels.multivariate.multivariate_ols._MultivariateOLSResults'>


In [17]:
print(statsmodels.multivariate.multivariate_ols._MultivariateOLSResults.__doc__)


    _MultivariateOLS results class

    Attributes
    ----------
    params : 2d ndarray
         coefficients of intercept (if fit) and exog variables for each endog variable
    df_resid : int
         residual degrees of freedom
    hasconst : boolean
         True if model was fit with a constant, else False.
    formula : str
         R-style formula from which model was constructed.
    ssr : float or array of floats
         residual sum of squares. See method docstring for additional info.
    df_model : int
         model degrees of freedom (including constant if fit)
    loglike : float
         Value of log-liklihood function.
    AIC : float
         Akaike's information criteria. See method docstring for additional info.
         


    


In [18]:
print(mod.exog_names)
print(noformula_mod.exog_names)
print(noConst_mod.exog_names)

['Intercept', 'mpg', 'cyl', 'disp']
['const', 'mpg', 'cyl', 'disp']
['mpg', 'cyl', 'disp']


In [19]:
print(mod.params)
print(noformula_mod.params)
print(mod.params == noformula_mod.params)

[[ 2.45534817e+01  4.92951014e+01]
 [-5.30029521e-02 -2.35788925e+00]
 [-1.15824380e+00  2.07718985e+01]
 [ 6.61733652e-03  7.03780053e-02]]
[[ 2.45534817e+01  4.92951014e+01]
 [-5.30029521e-02 -2.35788925e+00]
 [-1.15824380e+00  2.07718985e+01]
 [ 6.61733652e-03  7.03780053e-02]]
[[ True  True]
 [ True  True]
 [ True  True]
 [ True  True]]


In [20]:
# note that params includes the intercepts
print(noConst_mod.params.shape)
print(noConst_mod.params)

(3, 2)
[[ 5.59475931e-01 -1.12823844e+00]
 [ 6.41898127e-01  2.43859757e+01]
 [ 1.07804263e-02  7.87360840e-02]]


In [21]:
# this SHOULD include the intercept, since that is a parameter that is being estimated
print(mod.df_model)
print(noformula_mod.df_model)
print(noConst_mod.df_model)

4
4
3


In [22]:
# nobs - df_model
print(mod.df_resid)
print(noformula_mod.df_resid)
print(noConst_mod.df_resid)

28
28
29


In [23]:
print(mod.hasconst)
print(noformula_mod.hasconst)
print(noConst_mod.hasconst)

True
True
False


In [24]:
print(mod.formula)
print(noformula_mod.formula)
print(noConst_mod.formula)

qsec + hp ~ mpg + cyl + disp
qsec + hp ~ mpg + cyl + disp
qsec + hp ~ mpg + cyl + disp


In [25]:
print(mod.ssr())
print(noformula_mod.ssr())
print(noConst_mod.ssr()) # as expected, ssr is larger

21020.589940823113
21020.589940823113
21316.061165388273


In [26]:
print(mod.loglike)
print(noformula_mod.loglike)
print(noConst_mod.loglike)

-149.20638197340233
-149.20638197340233
-149.42971642369895


In [27]:
print(mod.aic)
print(noformula_mod.aic)
print(noConst_mod.aic) # lower due to lower model df

306.41276394680466
306.41276394680466
304.8594328473979


## Validate statistical calculations with univariate models:

In [28]:
import statsmodels.formula.api as smf
# using statsmodels multivariate OLS
univar_MOD = smf.ols(formula="hp ~ mpg + cyl + disp", data=cars)
univar_mod = univar_MOD.fit()

In [29]:
univar_mod.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     hp   R-squared:                       0.712
Model:                            OLS   Adj. R-squared:                  0.681
Method:                 Least Squares   F-statistic:                     23.06
Date:                Wed, 21 Oct 2020   Prob (F-statistic):           1.01e-07
Time:                        16:23:56   Log-Likelihood:                -160.27
No. Observations:                  32   AIC:                             328.5
Df Residuals:                      28   BIC:                             334.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     49.2951     87.723      0.562      0.579    -130.397     228.987
mpg           -2.3579      2.353     -1.002      0.325      -7.178       2.463
cyl           20.7719      9.764      2.127      0.042       0.771      40.772
disp           0.0704      0.139      0.507      0.616      -0.214       0.355
==============================================================================
Omnibus:                       20.455   Durbin-Watson:                   1.338
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               30.678
Skew:                           1.579   Prob(JB):                     2.18e-07
Kurtosis:                       6.611   Cond. No.                     3.36e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.36e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Validate log likelihood calculation:
from `statsmodels.regression.linear_model.OLS`:  
```python
def loglike(self, params, scale=None):
        """
        The likelihood function for the OLS model.
        Parameters
        ----------
        params : array_like
            The coefficients with which to estimate the log-likelihood.
        scale : float or None
            If None, return the profile (concentrated) log likelihood
            (profiled over the scale parameter), else return the
            log-likelihood using the given scale value.
        Returns
        -------
        float
            The likelihood function evaluated at params.
        """
        nobs2 = self.nobs / 2.0
        nobs = float(self.nobs)
        resid = self.endog - np.dot(self.exog, params)
        if hasattr(self, 'offset'): # what does this mean?
            resid -= self.offset
        ssr = np.sum(resid**2)
        if scale is None: # what does this mean?
            # profile log likelihood
            llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
        else:
            # log-likelihood
            llf = -nobs2 * np.log(2 * np.pi * scale) - ssr / (2*scale)
        return llf
```
Slightly modified for MultivariateOLS:  
* using externally calculated ssr
```python
def loglike(self, scale=None):
    nobs2 = self.nobs / 2.0
    nobs = float(self.nobs)
    ssr=self.ssr
    if scale is None: # what does this mean?
        # profile log likelihood
        llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
    else:
        # log-likelihood
        llf = -nobs2 * np.log(2 * np.pi * scale) - ssr / (2*scale)
    return llf
```

In [30]:
def loglike(modResults):
    ssr=modResults.ssr
    nobs=float(modResults.nobs)
    nobs2=nobs/2.0
    scale=modResults.scale
    if scale is None:
        # profile log likelihood
        llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
    else:
        # log-likelihood
        llf = -nobs2 * np.log(2 * np.pi * scale) - ssr / (2*scale)
    return llf

In [31]:
print(univar_mod.llf)
print(loglike(univar_mod))

-160.27451375704806
-160.41101603904042


Why are they slightly different? Offset maybe?

### Validate AIC calculation

In [32]:
# copy of formula from multivariate_ols.py, modify "self"
def aic(self):
        r"""                                                                                                                                                                                                                                                                    
        Akaike's information criteria.                                                                                                                                                                                                                                          
                                                                                                                                                                                                                                                                                
        :math:`-2llf + 2(df\_model)`.                                                                                                                                                                                                                                           
        Note that df_model includes the constant (if fit).                                                                                                                                                                                                                      
        """
        return -2 * self.llf + (2 * (self.df_model+1)) # add 1 for constant here, since in multivariate OLS I define model_df to include constant

In [33]:
univar_mod.aic

328.5490275140961

In [34]:
aic(univar_mod)

328.5490275140961

AIC calculation is correct.

## Validate model stats against R:

In [35]:
%load_ext rpy2.ipython

In [36]:
mod.formula

'qsec + hp ~ mpg + cyl + disp'

In [37]:
%%R -i cars
#print(class(cars))

mv1 <- lm(cbind(qsec, hp) ~ mpg + cyl + disp, data = cars)
print(extractAIC(mv1)) # this returns multiple values, guessing one for each response?
summary(mv1)

[1]  36.0000 487.2014
Response qsec :

Call:
lm(formula = qsec ~ mpg + cyl + disp, data = cars)

Residuals:
    Min      1Q  Median      3Q     Max 
-2.6385 -1.0082  0.1268  0.8874  3.2562 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 24.553482   3.270432   7.508 3.54e-08 ***
mpg         -0.053003   0.087735  -0.604  0.55062    
cyl         -1.158244   0.364015  -3.182  0.00356 ** 
disp         0.006617   0.005172   1.279  0.21123    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.444 on 28 degrees of freedom
Multiple R-squared:  0.4105,	Adjusted R-squared:  0.3474 
F-statistic:   6.5 on 3 and 28 DF,  p-value: 0.001777


Response hp :

Call:
lm(formula = hp ~ mpg + cyl + disp, data = cars)

Residuals:
   Min     1Q Median     3Q    Max 
-51.30 -22.08 -11.18  13.15 133.71 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)  
(Intercept) 49.29510   87.72268   0.562   0.5786  
mpg         -2.3

In [38]:
mod.params

array([[ 2.45534817e+01,  4.92951014e+01],
       [-5.30029521e-02, -2.35788925e+00],
       [-1.15824380e+00,  2.07718985e+01],
       [ 6.61733652e-03,  7.03780053e-02]])

## Validate implementation of F test
compare restricted vs un-restricted models

add as class to multivariate_ols.py

### Calculate the F statistic using residual sum of squares:
$$ F statistic = \frac{ \frac{RSS_1 - RSS_2}{k_2 - k_1} }{ \frac{RSS_2}{n-k_2} } $$
where:   
$RSS_1, RSS_2=\text{residual sum of squares for simple and complex models, respectively}$  
$k_1, k_2=\text{model degrees of freedom (number of independent/exog variables) in simple and complex model, respectively}$  
$n=\text{number of samples or observations}$.  
### Compare with sm.stats.anova_lm()
Set up two models:

In [39]:
m1 = smf.ols(formula="hp ~ mpg + cyl", data=cars).fit()
m2 = smf.ols(formula="hp ~ mpg + cyl + disp", data=cars).fit()

anovaRes = sm.stats.anova_lm(m1, m2)
anovaRes

,df_resid,ssr,df_diff,ss_diff,F,Pr(>F)
0,29.0,42368.731278,0.0,NaN,NaN,NaN
1,28.0,41982.827567,1.0,385.903711,0.257374,0.615902


Code from multivariate_ols.F_test_multivariate : 

```python
def F_statistic(self):
    """
    '1' refers to restricted model
    '2' refers to unrestricted model
    """
    num = (self.ssr1 - self.ssr2) / (self.df_model_2 - self.df_model_1)
    # NOTE: statsmodels.stats.anova_lm() has a "scale" parameter here. I'm not sure what that is.
    denom = self.ssr2 / (self.nobs - self.df_model_2)
    F_stat = num / denom
    return F_stat
   
def prF(self):
    df_diff = self.df_model_2 - self.df_model_1
    return stats.f.sf(self.F_stat, df_diff, self.df_resid)

def results(self):
    models=[self.restricted, self.unrestricted]
    cols=['formula','df_resid','ssr', 'model_df','model_df_diff','ssr_diff','F_stat','Pr>F']
    df = pd.DataFrame(np.zeros((2,len(cols))), index=["restricted model","unrestricted model"], 
                          columns=cols)
        
    df['formula'] = [mdl.formula for mdl in models]
    df['df_resid'] = [mdl.df_resid for mdl in models]
    df['ssr'] = [mdl.ssr for mdl in models]
    df['model_df'] = [mdl.df_model for mdl in models]
    df['model_df_diff'] = df['model_df'].diff()
    df['ssr_diff'] = df['ssr'].diff()
    df.loc['unrestricted model','F_stat'] = self.F_statistic
    df.loc['unrestricted model','Pr>F'] = self.prF
        
    df = df.replace(0, np.nan)
    return df
```

In [40]:
# modified to remove use of "self"
def F_statistic(m1, m2):
    """
    '1' refers to restricted model
    '2' refers to unrestricted model
    """
    num = (m1.ssr - m2.ssr) / (m2.df_model - m1.df_model)
    # NOTE: statsmodels.stats.anova_lm() has a "scale" parameter here. I'm not sure what that is.
    denom = m2.ssr / (m2.nobs - m2.df_model)
    F_stat = num / denom
    return F_stat

def prF(m1, m2, F_stat):
    df_diff = m2.df_model - m1.df_model
    return stats.f.sf(F_stat, df_diff, m2.df_resid)

def results(m1, m2, F_stat, prF):
    models=[m1, m2]
    cols=['formula','df_resid','ssr', 'model_df','model_df_diff','ssr_diff','F_stat','Pr>F']
    df = pd.DataFrame(np.zeros((2,len(cols))), index=["restricted model","unrestricted model"], 
                          columns=cols)

    #df['formula'] = [mdl.formula for mdl in models]
    df['df_resid'] = [mdl.df_resid for mdl in models]
    df['ssr'] = [mdl.ssr for mdl in models]
    df['model_df'] = [mdl.df_model for mdl in models]
    df['model_df_diff'] = df['model_df'].diff()
    df['ssr_diff'] = df['ssr'].diff()
    df.loc['unrestricted model','F_stat'] = F_stat
    df.loc['unrestricted model','Pr>F'] = prF

    df = df.replace(0, np.nan)
    return df

In [41]:
F_stat=F_statistic(m1, m2)
prF=prF(m1, m2, F_stat)
results(m1, m2, F_stat, prF)

,formula,df_resid,ssr,model_df,model_df_diff,ssr_diff,F_stat,Pr>F
restricted model,NaN,29.0,42368.731278,2.0,NaN,NaN,NaN,NaN
unrestricted model,NaN,28.0,41982.827567,3.0,1.0,-385.903711,0.266566,0.609698


In [42]:
anovaRes

,df_resid,ssr,df_diff,ss_diff,F,Pr(>F)
0,29.0,42368.731278,0.0,NaN,NaN,NaN
1,28.0,41982.827567,1.0,385.903711,0.257374,0.615902


In `statsmodels.stats.anova_lm`, the F statistic is divided by the "scale" property. This explains the difference. P value is close enough.

### Validate F test works with multivariate models:

In [43]:
mv1 = statsmodels.multivariate.multivariate_ols._MultivariateOLS.from_formula(
    formula="qsec + hp ~ mpg + cyl", data=cars).fit()
mv2 = statsmodels.multivariate.multivariate_ols._MultivariateOLS.from_formula(
    formula="qsec + hp ~ mpg + cyl + disp", data=cars).fit()

fTest = statsmodels.multivariate.multivariate_ols.F_test_multivariate(mv1, mv2)
print(fTest.F_statistic)
print(fTest.prF)

0.2592893842525494
0.6145984807321916


In [44]:
fTest.results

,formula,df_resid,ssr,model_df,model_df_diff,ssr_diff,F_stat,Pr>F
restricted model,qsec + hp ~ mpg + cyl,29,21215.247649,3,NaN,NaN,NaN,NaN
unrestricted model,qsec + hp ~ mpg + cyl + disp,28,21020.589941,4,1.0,-194.657708,0.259289,0.614598


### Compare F test calculation with R:
this is different, but I'm not sure the test is actually the same

In [45]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [46]:
%%R -i cars
print(class(cars))

mv1 <- lm(cbind(qsec, hp) ~ mpg + cyl, data = cars)
mv2 <- lm(cbind(qsec, hp) ~ mpg + cyl + disp, data = cars)
anova(mv1, mv2)

[1] "data.frame"
Analysis of Variance Table

Model 1: cbind(qsec, hp) ~ mpg + cyl
Model 2: cbind(qsec, hp) ~ mpg + cyl + disp
  Res.Df Df Gen.var.  Pillai approx F num Df den Df Pr(>F)
1     29      46.365                                      
2     28 -1   44.810 0.12924   2.0038      2     27 0.1544


## How to implement MANOVA
Test significance of each term in a multivariate model

In [47]:
manv = manova.MANOVA.from_formula(mod.formula, data=cars).mv_test()
#print(manv)

In [48]:
manv.summary_frame

Value Num DF Den DF  F Value       Pr > F
Effect    Statistic                                                           
Intercept Wilks' lambda           0.225702      2     27  46.3135  1.87377e-09
          Pillai's trace          0.774298      2     27  46.3135  1.87377e-09
          Hotelling-Lawley trace   3.43063      2     27  46.3135  1.87377e-09
          Roy's greatest root      3.43063      2     27  46.3135  1.87377e-09
mpg       Wilks' lambda           0.895814      2     27  1.57009     0.226435
          Pillai's trace          0.104186      2     27  1.57009     0.226435
          Hotelling-Lawley trace  0.116303      2     27  1.57009     0.226435
          Roy's greatest root     0.116303      2     27  1.57009     0.226435
cyl       Wilks' lambda           0.732924      2     27  4.91936    0.0150765
          Pillai's trace          0.267076      2     27  4.91936    0.0150765
          Hotelling-Lawley trace  0.364397      2     27  4.91936    0.0150765
          Roy's greatest root     0.364397      2     27  4.91936    0.0150765
disp      Wilks' lambda           0.870757      2     27  2.00376     0.154386
          Pillai's trace          0.129243      2     27  2.00376     0.154386
          Hotelling-Lawley trace  0.148426      2     27  2.00376     0.154386
          Roy's greatest root     0.148426      2     27  2.00376     0.154386

### Compare statsmodels MANOVA with R:
Some terms are different- why?

In [49]:
%%R -i cars

mv1 <- lm(cbind(qsec, hp) ~ mpg + cyl + disp, data = cars)
anova(mv1)

Analysis of Variance Table

            Df  Pillai approx F num Df den Df    Pr(>F)    
(Intercept)  1 0.99749   5359.5      2     27 < 2.2e-16 ***
mpg          1 0.69228     30.4      2     27 1.231e-07 ***
cyl          1 0.30834      6.0      2     27  0.006896 ** 
disp         1 0.12924      2.0      2     27  0.154386    
Residuals   28                                             
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


## Forward stepwise model selection
1. Start with null model (Y ~ 1)
2. Fit p OLS regression models, each with one of the X variables  and the intercept
3. Select the best out of these models, and fix this X term in the model going forward.
  * lowest residual sum of squares
  * R squared?
  * P-value?
4. Search through  remaining p-1 variables and determine which variable should be added to the current model to best improve the residual sum of squares.
5. Continue until some stopping rule is satisfied
  * additional terms have p-value > 0.05
  * minimum AIC is reached

In [50]:
null = statsmodels.multivariate.multivariate_ols._MultivariateOLS.from_formula(
    formula="qsec + hp ~ 1", data=cars).fit()

In [51]:
null.endog_names

['qsec', 'hp']

In [52]:
null.exog_names

['Intercept']

```python
# TODO: add r2 calculation
idx=pd.IndexSlice
def forward_stepwise(X, Y, data=None, param_select="ssr", statistic="pillai", verbose=False):
    """
    Performs forward stepwise model selection for a set of X and Y data.
    
    Parameters
    -------
    X : array-like
        Dataframe or array containing independent/exog variable data. Must be of
        shape (m,n) where m is variables and n is observations.
        If data is not None, must be list-like, corresponding to column names in data.
    Y : array-like
        Dataframe or array containing dependent/endog variable data. Must be of
        shape (m,n) where m is outcome(s) and n is observations.
        If data is not None, must be list-like, corresponding to column names in data.
    data : array-like, optional
        Dataframe containing both independent/exog and dependent/endog variables. Must
        be of shape (m,n), where m is variables/outcomes and n is observations.
    param_select : str, optional
        Can be one of ['ssr', 'pval']. Metric used to select the parameter that
        best improves the model at each iteration. Default is 'ssr'.
    statistic : str, optional
        Can be one of ['pillai','wilks','hotelling-lawly','roys']. Test statistic used by
        statsmodels.multivariate.manova to calculate statistical significance of added parameters.
    verbose : boolean, optional
        If True, return an additional dataframe containing information about paramters that
        were not added to model due to co-linearity.
    
    Returns
    -------
    results : dataframe
        Dataframe of model parameters at each iteration of selection
    dropped_params : dataframe, optional
        Only returned if verbose=True.
    
    """
    
    if data is not None:
        endog=data[Y]
        endog_names=Y
        exog=data[X]
        exog_names=X
    else:
        raise NotImplementedError("use with 'data' arg for now.")
        endog=Y
        endog_names=Y.columns
        exog=X
        exog_names=X.columns
        # combine data
        if isinstance(X, pd.DataFrame):
            data = pd.concat([X,Y], axis=1, ignore_index=True)
        if isinstance(X, np.ndarray): # TODO: fix.
            raise NotImplementedError("X and Y should be dataframes for now.")
    
    # check if all data is numeric
    numeric = data.apply(lambda s: pd.to_numeric(s, errors='coerce').isnull().all())
    if numeric.sum() > 0:
        fail = numeric.index[numeric==True].values
        msg = "Data in the following column(s) is not numeric! {0}".format(str(fail))
        raise ValueError(msg)
    numeric.index[numeric==True].values
    
    # parse "statistics" arg:
    stats_dict = dict(zip(['pillai','wilks','hotelling-lawly','roys'],
                         ["Pillai's trace","Wilks' lambda","Hotelling-Lawley trace","Roy's greatest root"]))
    manv_statistic = stats_dict[statistic]
    
    
    # initialize empty dataframes to store results from each round of parameter addition, and dropped params
    cols=["formula","ssr","log-likelihood","F statistic","Pr>F","AIC"]
    resDF = pd.DataFrame(np.zeros((1,len(cols))), index=["NullModel"], columns=cols)
    
    dropDF = pd.DataFrame(columns=["colinear_var","spearman_r"])
    
    # initialize null model:
    nullformula = "{0} ~ 1".format(" + ".join(endog_names))
    nullmod = statsmodels.multivariate.multivariate_ols._MultivariateOLS.from_formula(
    formula=nullformula, data=data).fit()
    # add to results df
    resDF.loc["NullModel"] = [nullmod.formula, nullmod.ssr(), nullmod.loglike, np.nan, np.nan, nullmod.aic]
    
    #import pdb; pdb.set_trace() #####
    
    params=exog_names
    bestmodel=None
    while len(params) > 0: # e.g. as long as there are parameters remaining
        
        # initialize empty df to store parameter data
        cols=["paramter","ssr","statistic","F-value","Pr>F"]
        paramDF = pd.DataFrame(columns=cols, index=params)
        
        for param in params:
            if bestmodel: # defined at end of first iteration
                refmodel=bestmodel
            else:
                refmodel=nullmod
            
            # build + fit model with parameter:
            formula = refmodel.formula + " + {0}".format(str(param))
            testmod = statsmodels.multivariate.multivariate_ols._MultivariateOLS.from_formula(
                formula=formula, data=data).fit()
            
            # run MANOVA to calculate p val for parameters
            manv = manova.MANOVA.from_formula(testmod.formula, data=cars).mv_test()
            Fval = manv.summary_frame.loc[idx[param, manv_statistic],"F Value"]
            PrF = manv.summary_frame.loc[idx[param, manv_statistic],"Pr > F"]
            
            # add data to paramDF
            paramDF.loc[param] = [param,testmod.ssr(),manv_statistic,Fval,PrF]
            ########################## end of parameter addition ###########################
        
        # Choose best parameter:
        if param_select == "ssr":
            bestParam = paramDF.sort_values(by="ssr", ascending=True).index[0]
        elif param_select == "pval":
            bestParam = paramDF.sort_values(by="Pr>F", ascending=True).index[0]
        else:
            raise ValueError("'param_select' must be either 'ssr' or 'pval'. Got {0}".format(str(param_select)))
        
        # initialize model with best parameter
        formula = refmodel.formula + " + {0}".format(str(bestParam))
        bestmodel = statsmodels.multivariate.multivariate_ols._MultivariateOLS.from_formula(
                formula=formula, data=data).fit()
            
        # run F test versus refmodel:
        # arg order: refmod, bestmod 
        fTest = statsmodels.multivariate.multivariate_ols.F_test_multivariate(refmodel, bestmodel)
        
        # add data to results df
        resDF.loc[bestParam] = [bestmodel.formula, bestmodel.ssr(), bestmodel.loglike,
                                fTest.F_statistic, fTest.prF, bestmodel.aic]
        
        # remove bestParam and any highly co-linear variables from remaining params:
        corr = data[params].corr(method='spearman')
        drop = list(corr[abs(corr[bestParam]) > 0.8].index) # this includes the term itself since r=1
        [params.remove(d) for d in drop]
        # note in resDF that parameter was dropped due to co-linearity
        drop.remove(bestParam)
        
        for d in drop:
            dropDF.loc[d,"colinear_var"] = bestParam
            dropDF.loc[d,"spearman_r"] = corr.loc[d,bestParam]
            #resDF.loc[d,"formula"] = "dropped due to colinearity with {0}".format(bestParam)
    if verbose == True:
        return resDF.sort_values(by="AIC", ascending=True), dropDF
    else:
        return resDF.sort_values(by="AIC", ascending=True)
            
#forward_stepwise(X=['mpg', 'cyl', 'disp', 'drat', 'wt', 'vs', 'am', 'gear','carb'], 
#                 Y=['qsec','hp'], data=cars)
```

In [53]:
l=[1,2,3,4,5]
while len(l) > 0:
    print("starting for loop")
    for num in l:
        print(num)
    l.remove(max(l))
    

starting for loop
1
2
3
4
5
starting for loop
1
2
3
4
starting for loop
1
2
3
starting for loop
1
2
starting for loop
1


In [54]:
import statsmodels.multivariate.modelselection
res, dropped = statsmodels.multivariate.modelselection.forward_stepwise(
    X=['mpg', 'cyl', 'disp', 'drat', 'wt', 'vs', 'am', 'gear','carb'], 
                 Y=['qsec','hp'], data=cars, return_dropped=True, verbose=True)
res

Iteration #1
adding cyl to model
dropped: mpg, disp, wt, vs
-------------------- 

Iteration #2
adding carb to model
dropped: 
-------------------- 

Iteration #3
adding gear to model
dropped: am
-------------------- 

Iteration #4
adding drat to model
dropped: 
-------------------- 



,formula,param_Pr>F,ssr,log-likelihood,df_model,F statistic,Pr>F,AIC
carb,cyl + carb,1.045261e-04,12630.574367,-141.056270,3.0,22.438894,5.267142e-05,288.112539
gear,cyl + carb + gear,2.466166e-03,12024.929622,-140.270054,4.0,1.410241,2.449932e-01,288.540109
drat,cyl + carb + gear + drat,3.576686e-01,12009.092108,-140.248968,5.0,0.035607,8.517397e-01,290.497935
cyl,cyl,3.664634e-08,22403.544024,-150.225850,2.0,67.635800,3.518220e-09,304.451699
NullModel,,NaN,72912.931575,-169.106599,1.0,NaN,NaN,340.213199
